In [16]:
from pathlib import Path
import open3d as o3d
import os

from pytorch_lightning import seed_everything

from src.dataset_utils import (
    get_singleview_data,
    get_multiview_data,
    get_voxel_data_json,
    get_image_transform_latent_model,
    get_pointcloud_data,
    get_mv_dm_data,
    get_sv_dm_data,
    get_sketch_data
)
from src.model_utils import Model
from src.mvdream_utils import load_mvdream_model
import argparse
from PIL import Image


In [17]:
from comet_ml import start
from comet_ml.integration.pytorch import log_model

experiment = start(
  api_key="mqrUAXjKBRul24uX6pxR3gRHX*eyJiYXNlVXJsIjoiaHR0cHM6Ly9jb21ldC5kZXYuY2xvdWRvcy5hdXRvZGVzay5jb20ifQ",
  project_name="wala-time-checks",
  workspace="alessandro-giuliano"
)

COMET INFO: An experiment with the same configuration options is already running and will be reused.


In [18]:

def simplify_mesh(obj_path, target_num_faces=1000):
    mesh = o3d.io.read_triangle_mesh(obj_path)
    simplified_mesh = mesh.simplify_quadric_decimation(target_num_faces)
    o3d.io.write_triangle_mesh(obj_path, simplified_mesh)


def generate_3d_object(
    model,
    data,
    data_idx,
    scale,
    diffusion_rescale_timestep,
    save_dir="examples",
    output_format="obj",
    target_num_faces=None,
    seed=42,
    
):
    # Set seed
    seed_everything(seed, workers=True)

    save_dir.mkdir(parents=True, exist_ok=True)
    model.set_inference_fusion_params(scale, diffusion_rescale_timestep)
    output_path = model.test_inference(
        data, data_idx, experiment, save_dir=save_dir, output_format=output_format
    )

    if output_format == "obj" and target_num_faces:
        simplify_mesh(output_path, target_num_faces=target_num_faces)


In [19]:
model_name = 'ADSKAILab/WaLa-SV-1B'
images_path = Path('examples/single_view/')
single_image = ['examples/single_view/table.png']

output_dir = 'examples/Test_Gen'
output_format = 'obj'
target_num_faces = None
scale = 1.8
seed = 42
diffusion_rescale_timestep = 5



In [20]:
print(f"Loading model")

model = Model.from_pretrained(pretrained_model_name_or_path=model_name)
image_transform = get_image_transform_latent_model()

for image_path in single_image:
    print(f"Processing image: {image_path}")
    data = get_singleview_data(
        image_file=Path(image_path),
        image_transform=image_transform,
        device=model.device,
        image_over_white=False,
    )
    data_idx = 0
    save_dir = Path(output_dir) / Path(image_path).stem

    model.set_inference_fusion_params(
        scale, diffusion_rescale_timestep
    )

    generate_3d_object(
        model,
        data,
        data_idx,
        scale,
        diffusion_rescale_timestep,
        save_dir,
        output_format,
        target_num_faces,
        seed
    )


    

Loading model
'DotDict' object has no attribute 'dataset_path'
'DotDict' object has no attribute 'low_avg'
'DotDict' object has no attribute 'low_avg'
Low avg used : None high value: 63


/opt/miniconda/envs/wala/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.1.0
Using cache found in /home/ray/.cache/torch/hub/facebookresearch_dinov2_main
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


cond_emb_dim: 1024
Input resolution: 224
Vocab size: N/A
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


Seed set to 42


Processing image: examples/single_view/table.png
Extract Image 0.12564563751220703 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.402419090270996 s
Latent Decoding Time 0.0975334644317627 s
Wavelet Preparation Time 0.0002522468566894531 s
Low to Highs conversion 0.2680997848510742 s
Inverse DWT time elapsed 0.010677576065063477 s
mcubes.marching_cubes time 0.5867412090301514 s
export obj time 2.3584489822387695 s
Time to save visualization from high lows to obj 2.9565439224243164 s


In [6]:
import time


for idx, image_path in enumerate(images_path.iterdir()):  # Enumerate to get the index
    timer1 = time.time()
    print(f"Processing image: {image_path}")
    data = get_singleview_data(
        image_file=Path(image_path),
        image_transform=image_transform,
        device=model.device,
        image_over_white=False,
    )
    data_idx = 0
    save_dir = Path(output_dir) 

    model.set_inference_fusion_params(
        scale, diffusion_rescale_timestep
    )

    generate_3d_object(
        model,
        data,
        data_idx,
        scale,
        diffusion_rescale_timestep,
        save_dir,
        output_format,
        target_num_faces,
        seed,
    )
    delta = time.time() - timer1
    experiment.log_metric("Default Delta", delta)
    print('Total Inference time', delta)

experiment.end()

Seed set to 42


Processing image: examples/single_view/table.png
Extract Image 0.016147613525390625 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.348755121231079 s
Latent Decoding Time 0.005625724792480469 s
Wavelet Preparation Time 0.00020551681518554688 s
Low to Highs conversion 0.27273058891296387 s
Inverse DWT time elapsed 0.0033140182495117188 s
mcubes.marching_cubes time 0.5861926078796387 s


Seed set to 42


...

export obj time 2.0231595039367676 s
Time to save visualization from high lows to obj 2.5854058265686035 s
Total Inference time 5.2440409660339355
Processing image: examples/single_view/pear.jpeg
Extract Image 0.014765024185180664 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.3535726070404053 s
Latent Decoding Time 0.004456758499145508 s
Wavelet Preparation Time 0.00014972686767578125 s
Low to Highs conversion 0.24218201637268066 s
Inverse DWT time elapsed 0.0028891563415527344 s
mcubes.marching_cubes time 0.5516839027404785 s


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : fashionable_durian_9726
COMET INFO:     url                   : https://comet.dev.cloudos.autodesk.com/alessandro-giuliano/wala-time-checks/b8a38b6c4e3749158d63843fccbaea46
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Default Delta [3]                                    : (4.964773178100586, 5.6073362827301025)
COMET INFO:     Extract Image [4]                                    : (0.014613866806030273, 0.12229490280151367)
COMET INFO:     Inverse DWT time elapsed [4]                         : (0.0028891563415527344, 0.011530399322509766)
COMET INFO:     Latent Decoding Time [4]                             : (0.0043375492095947266, 0.084

export obj time 1.786191463470459 s
Time to save visualization from high lows to obj 2.3415040969848633 s
Total Inference time 4.964773178100586


## Dataset Extraction and Utilization for Benchmarking

### Dataset download and extraction

In [ ]:
import zipfile
from pathlib import Path
import os
import shutil

zip_dir = '/home/ray/WaLa/Raw_data'         # Folder with zip files
output_dir = '/home/ray/WaLa/test_images'  # Where to save extracted thumbnails
os.makedirs(output_dir, exist_ok=True)

for zip_path in Path(zip_dir).glob('*.zip'):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # Find files in the 'thumbnail' folder (not subfolders)
        thumbnail_files = [f for f in z.namelist() if f.startswith('thumbnails/') and not f.endswith('/')]
        if thumbnail_files:
            thumb_file = thumbnail_files[0]  # Pick the first one
            ext = Path(thumb_file).suffix
            out_name = zip_path.stem + ext
            out_path = Path(output_dir) / out_name
            with z.open(thumb_file) as source, open(out_path, 'wb') as target:
                shutil.copyfileobj(source, target)
            print(f"Extracted {thumb_file} from {zip_path.name} as {out_name}")

In [ ]:
from pathlib import Path

def count_extracted_images(target_folder, extensions={'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}):
    """
    Counts the number of image files in the target folder.
    Args:
        target_folder (str or Path): Path to the folder to check.
        extensions (set): Set of file extensions to consider as images.
    Returns:
        int: Number of image files found.
    """
    target_folder = Path(target_folder)
    return sum(1 for f in target_folder.iterdir() if f.suffix.lower() in extensions and f.is_file())

num_images = count_extracted_images('/home/ray/WaLa/test_images')  # Where to save extracted thumbnails
print(f"Extracted images: {num_images}")

Extracted images: 0


### Dataset Inference Tests

In [25]:
from comet_ml import start
from comet_ml.integration.pytorch import log_model
import time

experiment = start(
  api_key="mqrUAXjKBRul24uX6pxR3gRHX*eyJiYXNlVXJsIjoiaHR0cHM6Ly9jb21ldC5kZXYuY2xvdWRvcy5hdXRvZGVzay5jb20ifQ",
  project_name="wala-time-checks",
  workspace="alessandro-giuliano")


  # Define the hyperparameters
hyperparameters = {
    "model_name": "ADSKAILab/WaLa-SV-1B",
    "images_path": str(Path("examples/single_view/")),
    "single_image": ["examples/single_view/table.png"],
    "output_dir": "examples/Test_Gen",
    "output_format": "obj",
    "target_num_faces": None,
    "scale": 1.8,
    "seed": 42,
    "diffusion_rescale_timestep": 5
}

# Log the hyperparameters to Comet
experiment.log_parameters(hyperparameters)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://comet.dev.cloudos.autodesk.com/alessandro-giuliano/wala-time-checks/1bf42d8c3ee545628968903512893acf



In [ ]:
### Scanned Objects by Google Research
google_test_img_path = Path(os.path.expanduser('~/test_images'))  # Expands '~' to the full home directory path
output_dir = os.path.expanduser('~/Obj_generated')

for idx, image_path in enumerate(google_test_img_path.iterdir()):  # Enumerate to get the index
        #if idx >= 3:  # Stop after processing the first 3 files
            #break
        print(f"Processing image {idx + 1}: {image_path}")



        timer1 = time.time()
        data = get_singleview_data(
            image_file=Path(image_path),
            image_transform=image_transform,
            device=model.device,
            image_over_white=False,
        )
        data_idx = 0
        save_dir = Path(output_dir) 

        model.set_inference_fusion_params(
            scale, diffusion_rescale_timestep
        )

        generate_3d_object(
            model,
            data,
            data_idx,
            scale,
            diffusion_rescale_timestep,
            save_dir,
            output_format,
            target_num_faces,
            seed,
        )
        delta = time.time() - timer1
        experiment.log_metric("Default Delta", delta)
        print('Total Inference time', delta)

experiment.end()

Seed set to 42


Processing image 1: /home/ray/test_images/2_of_Jenga_Classic_Game.jpg
Extract Image 0.015231847763061523 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.3450021743774414 s
Latent Decoding Time 0.00432586669921875 s
Wavelet Preparation Time 0.00014734268188476562 s
Low to Highs conversion 0.055942535400390625 s
Inverse DWT time elapsed 0.0028400421142578125 s
mcubes.marching_cubes time 0.5419051647186279 s


Seed set to 42


...

Latent Diffusion Time 2.3599159717559814 s
Latent Decoding Time 0.004387617111206055 s
Wavelet Preparation Time 0.00016307830810546875 s
Low to Highs conversion 0.055947065353393555 s
Inverse DWT time elapsed 0.002892732620239258 s
mcubes.marching_cubes time 0.5592677593231201 s


Seed set to 42


export obj time 4.124996900558472 s
Time to save visualization from high lows to obj 4.68790078163147 s
Total Inference time 7.133396148681641
Processing image 218: /home/ray/test_images/Cole_Hardware_Butter_Dish_Square_Red.jpg
Extract Image 0.015001058578491211 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.358747720718384 s
Latent Decoding Time 0.004492998123168945 s
Wavelet Preparation Time 0.0001780986785888672 s
Low to Highs conversion 0.05602097511291504 s
Inverse DWT time elapsed 0.0028448104858398438 s
mcubes.marching_cubes time 0.5553078651428223 s
